# Loading and concatenante data 

In [ ]:
import numpy as np
import scipy.io


from sklearn.preprocessing import StandardScaler
import copy

# sparse_files = ['sparse_ANM210861_20130701.npy', 'sparse_ANM210861_20130702.npy', 
#                'sparse_ANM210861_20130703.npy']

# label_files = ['ANM210861_20130701_labels.npy', 'ANM210861_20130702_labels.npy',
#               'ANM210861_20130703_labels.npy']

sparse_files = ['sparse_ANM210861_20130701.npy', 'sparse_ANM210861_20130702.npy', 
               'sparse_ANM210861_20130703.npy', 'sparse_ANM210862_20130626.npy', 'sparse_ANM210862_20130627.npy', 
               'sparse_ANM210862_20130628.npy']

label_files = ['ANM210861_20130701_labels.npy', 'ANM210861_20130702_labels.npy',
              'ANM210861_20130703_labels.npy', 'ANM210862_20130626_labels.npy', 'ANM210862_20130627_labels.npy',
              'ANM210862_20130628_labels.npy']


agg_spares_epochs = np.load(sparse_files[0])
agg_labels = np.load(label_files[0])
ind = np.linspace(0,3443-1, num = 1033)
ind = [int(np.floor(i)) for i in ind]
agg_spares_epochs = agg_spares_epochs[:,:, ind]


for i in range(1,6):
    new_epochs =  np.load(sparse_files[i])
    agg_spares_epochs = np.concatenate((agg_spares_epochs, new_epochs), axis = 0)
    new_label = np.load(label_files[i])
    agg_labels = np.concatenate((agg_labels, new_label), axis = 0)

print(agg_spares_epochs.shape)
print(agg_labels.shape)



agg_epochs_std = copy.copy(agg_spares_epochs)
sample_num, chan_num, timepoint = agg_spares_epochs.shape
for c in range(chan_num):
    original_timepoints = agg_spares_epochs[:,c,:]
    scaler = StandardScaler()
    scaler.fit(original_timepoints)
    chan_std = scaler.transform(original_timepoints)
    agg_epochs_std[:,c,:] = chan_std


# Down sampling with logistic regression

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from pyriemann.tangentspace import FGDA, TangentSpace

ind = np.linspace(0,1032, num = 1032)
ind = [int(np.floor(i)) for i in ind]
epochs_ds = agg_epochs_std[:,:,ind]




cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs_ds
labels = agg_labels

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    ts = TangentSpace()
    T = ts.fit_transform(cov_X_train, y_train)
    
#     clf = sklearn.linear_model.LogisticRegression()
    clf=SVC(kernel='poly', degree = 2.7,  random_state=0, coef0 =0.2, gamma= 0.03, C=100)
    clf.fit(T, y_train)

    
    
    T_test = ts.fit_transform(cov_X_test, y_test)
    
    y_predict = clf.predict(T_test)
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# logistic with PCA

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft
from sklearn.decomposition import PCA


from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from pyriemann.tangentspace import FGDA, TangentSpace

ind = np.linspace(0,1032, num = 900)
ind = [int(np.floor(i)) for i in ind]
epochs_ds = agg_epochs_std[:,:,ind]




cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs_ds
labels = agg_labels

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    ts = TangentSpace()
    T = ts.fit_transform(cov_X_train, y_train)
    print(T.shape)
    
    pca = PCA(n_components=200)
    T_pca = pca.fit_transform(T)
    print(T_pca.shape)

    
    clf = sklearn.linear_model.LogisticRegression()
    clf.fit(T_pca, y_train)

    
    
    T_test = ts.fit_transform(cov_X_test, y_test)
    print(T_test.shape)

    pca = PCA(n_components=200)
    T_test_pca = pca.fit_transform(T_test)
    
    
    y_predict = clf.predict(T_test_pca)
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# Tangent Space classifier with single SVM

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import timeit





cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = agg_epochs_std
labels = agg_labels


# degree = [1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6]
# degree = [2.7, 2.8, 2.9, 3.0, 3.1, 3.2,3.3, 3.4, 3.5, 3.6]
degree = np.linspace(3.7, 5.5, 19)
coefs = np.linspace(0.0, 0.5, 51)


for coef in coefs:
    print(coef)
    for train_idx, test_idx in cv.split(epochs_data):
        start = timeit.default_timer()
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = epochs_data[train_idx]
        X_test = epochs_data[test_idx]


        cov =  pyriemann.estimation.Covariances('lwf')
        cov_X_train = cov.transform(X_train)
        cov_X_test = cov.transform(X_test)

    #     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=sklearn.discriminant_analysis.LinearDiscriminantAnalysis())
    #     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100))
        TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='poly', degree = 2.7,  random_state=0, coef0 =coef, gamma= 0.03, C=100))

    #     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=LogisticRegression())

        TSclassifier.fit(cov_X_train, y_train)


        y_predict = TSclassifier.predict(cov_X_test)
        print(sklearn.metrics.accuracy_score(y_predict, y_test))
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))
        stop = timeit.default_timer()
        print('Time: ', stop - start)  


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))



#non-even number of degree
# degree =2  70%
# degree = 2.2     0.776347
#2.6     0.776347 ON AVERAGE
#    2.2 with 0.2 coef   => 0.786228!!!    with 0.3 coef 0.784431;    with 0.1 =>0.784731

    
#linear with 0.2 coef =>   0.701796

#   degree 1.2  coef 0.2   =>  0.724251

# degree 1.4   coef  0.2  => 0.724251

# degree 1.6          0.724251

#1.8          0.724251
#1.9           0.724251
#2.0         0.744910
#2.1           0.755240 
#2.2          0.761437
#2.3          0.765569 
#2.4           0.768520
#2.5            0.770734
#2.6           0.772455
#2.7           0.786228
#2.8            0.786228
#2.9          0.786228
#3               0.782859
#3.1           0.780838
#3.2              0.779491
#3.3              0.778529
#3.4           0.777807
#3.5           0.777246
#3.6            0.776796
#3.7            0.772754

#3.8               0.772754
#3.9              0.772754
#4.0            0.772231
#                      0.771916
#                  0.771707 
#                 0.771557
#                 0.771445 
#                 0.771357 
#                 0.771287
#                 0.771230
#                 0.771183
#                 0.771142
#                 0.770488
#      degree 5.1    0.769920
#                   0.769424
#                    0.768986
#                   0.768596
#                 0.768248

In [ ]:
# degree = np.linspace(3.7, 5.5, 19)
# print(degree)

coefs = np.linspace(0.5, 5, 46)
print(coefs)

# Bagging of 21 SVM base classifiers with fine-tuned parameters

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []




n_estimators = [9,11,13]
n_samples, _,_ = agg_epochs_std.shape
labels = agg_labels

agg_epochs_std_train_test = agg_epochs_std
epochs_data = agg_epochs_std_train_test

for n_estimator in n_estimators:
    for train_idx, test_idx in cv.split(epochs_data):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = epochs_data[train_idx]
        X_test = epochs_data[test_idx]
        print(X_train.shape)
        print(X_test.shape)


        cov =  pyriemann.estimation.Covariances('lwf')
        cov_X_train = cov.transform(X_train)
        cov_X_test = cov.transform(X_test)

        n_samples_train, _,_ = cov_X_train.shape
        subset_size = n_samples_train//3 * 2

        ind_sets = [np.random.randint(0, n_samples_train, size = subset_size) for i in range(n_estimator)]

        clfs = [pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', degree=2.7, coef0=0.2,  random_state=0, gamma= 0.03, C=100, probability=True)) for i in range(n_estimator)]


        for i in range(n_estimator):
            clfs[i].fit(cov_X_train[ind_sets[i],:,:], y_train[ind_sets[i]])

        y_predict = [clfs[i].predict(cov_X_test) for i in range(n_estimator)]
        y_predict = scipy.stats.mode(y_predict, axis=0).mode[0]


        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))
        print(sklearn.metrics.accuracy_score(y_predict, y_test))



    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))


# Prepare for two stage classifier

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

cv = ShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
scores = []




n_estimator = 9
n_samples, _,_ = agg_epochs_std.shape
labels = agg_labels

agg_epochs_std_train_test = agg_epochs_std
epochs_data = agg_epochs_std_train_test

wrong = []

for train_idx, test_idx in cv.split(epochs_data):
    val_idx = test_idx[len(test_idx)//2:]
    test_idx = test_idx[:len(test_idx)//2]
    X_validation = epochs_data[val_idx]
    y_validation = np.asarray(labels)[val_idx]
    
    print(len(val_idx))
    
    
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)
    print(X_test.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
#     n_samples_train, _,_ = cov_X_train.shape
#     subset_size = n_samples_train//3 * 2
    
#     ind_sets = [np.random.randint(0, n_samples_train, size = subset_size) for i in range(n_estimator)]
    
#     clfs = [pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True)) for i in range(n_estimator)]
    clf = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True))
#     clf = pyriemann.classification.TSclassifier(metric='riemann')

#     for i in range(n_estimator):
#         clfs[i].fit(cov_X_train[ind_sets[i],:,:], y_train[ind_sets[i]])
    
#     y_predict = [clfs[i].predict(cov_X_test) for i in range(n_estimator)]
#     y_predict = scipy.stats.mode(y_predict, axis=0).mode[0]

    clf.fit(cov_X_train, y_train)
    y_predict = clf.predict(cov_X_test)

    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    
    
#     wrong.append([test_idx[i] for i in range(len(y_test)) if y_predict[i] != y_test[i]])

class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))


In [ ]:
print(np.mean(scores))

Pick out the channel that were repeatedly classified wrong

In [ ]:
result = [1128, 1268, 833, 1090, 983, 608, 625, 1201, 602, 901, 1061, 1117, 777, 785, 1295, 680, 903, 724, 826, 296, 1178, 1155, 675, 470, 771, 744, 247, 554, 295, 715, 656, 1051, 1243, 369, 490, 858, 1143, 879, 643, 1272, 1209, 931, 505, 835, 676, 580, 1286, 1324, 1127, 21, 1033, 765, 576, 738, 1180, 760, 670, 81, 1222, 1185, 1182, 839, 743, 723, 254, 429, 936, 1115, 716, 471, 315, 1126, 463, 1186, 184, 244, 1179, 682, 1109, 283, 704, 1247, 1257, 485, 1314, 1057, 1005, 944, 425, 472, 1104, 996, 1161, 1080, 834, 577, 975, 1015, 1008, 1176, 825, 1192, 1083, 1248, 1320, 898, 790, 503, 836, 831, 1184, 1242, 1274, 371, 650, 1141, 655, 1079, 1043, 838, 1211, 755, 436, 874, 1029, 1053, 518, 719, 599, 491, 1147, 997, 1049, 1167, 904, 811, 803, 1157, 1018, 872, 354, 638, 717, 1082, 991, 837, 1025, 37, 1170, 805]
result = result + sum(wrong, [])
from collections import Counter
l = [i[0] for i in list(Counter(result).most_common(150))]
print(l)



# Training another classifier to classify "good" epochs and "bad" epochs

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from pyriemann.tangentspace import FGDA, TangentSpace

correct = [i for i in np.random.randint(0,(n_samples//5 * 4),500) if i not in l]
bad = l
ind = correct + bad
X = agg_epochs_std_train_test[ind]
y = [0 for i in range(len(correct))] + [1 for i in range(len(bad))] 


cv = ShuffleSplit(n_splits=2, test_size=0.2, random_state=42)
scores = []
epochs_data = X
labels = y


inter_adj = [0, 1.1526818432767743, 0.4054674414465533, -0.057156080501559604, -0.4054627747697755, -0.6931448472215563, -0.9444592755024624, -1.1727179274834425, -1.3862920277815018, -1.591086440427515, -1.791757135889666, -1.9924278313518173]

for adj in inter_adj:
    print(adj)
    for train_idx, test_idx in cv.split(epochs_data):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = epochs_data[train_idx]
        X_test = epochs_data[test_idx]
        
        print(X_test.shape)


        cov =  pyriemann.estimation.Covariances('lwf')
        cov_X_train = cov.transform(X_train)
        cov_X_test = cov.transform(X_test)


        ts = TangentSpace()
        ts.fit(cov_X_train, y_train)
        T = ts.transform(cov_X_train)


        bad_catcher =LogisticRegression(penalty='l2', C=1.0, fit_intercept=True)
        bad_catcher.fit(T, y_train)


        bad_catcher.intercept_[0] = bad_catcher.intercept_[0] - adj
        
        
        T_test = ts.transform(cov_X_test)

        y_predict = bad_catcher.predict(T_test)
        
        
#         print(sklearn.metrics.accuracy_score(y_predict, y_test))
        true_negative, false_negative, false_positive, true_positive = confusion_matrix(y_test,y_predict)[0][0],confusion_matrix(y_test,y_predict)[1][0],confusion_matrix(y_test,y_predict)[0][1],confusion_matrix(y_test,y_predict)[1][1]   
        recall = (true_positive/(true_positive+false_negative))
        specificity = (true_negative/(true_negative+false_positive))
        print(confusion_matrix(y_test,y_predict))
        print("recall: {}".format(recall))
        print("specificiity: {}".format(specificity))
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))

    

# Two stag classifer on validation data

In [ ]:
def check_good(epochs, threshold = 0.65):
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_check_bad= cov.transform(epochs)
    T_test = ts.transform(cov_check_bad)
    bad = bad_catcher.predict(T_test)
    bad_proba = bad_catcher.predict_proba(T_test)
    for i in range(len(bad)):
        if bad[i] == 1: 
            if bad_proba[i,1] < threshold:
                bad[i] = 0
                
    print(list(bad).count(1))
    res = [i for i in range(epochs.shape[0]) if bad[i] == 0]
    return res



threshold = [1,0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55]

for t in threshold:
    print(X_validation.shape)
    ind = check_good(X_validation, t)


    X_validation_ind = X_validation[ind]
    print("reminding: {}".format(X_validation_ind.shape[0] / X_validation.shape[0]))

    y_validation_ind = y_validation[ind]



    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_validation= cov.transform(X_validation_ind)



    y_predict = clf.predict(cov_X_validation)


    print(sklearn.metrics.accuracy_score(y_predict, y_validation_ind))


Appendix:
    


[(1128, 34), (1268, 32), (1090, 28), (983, 28), (608, 28), (625, 28), (833, 27), (1201, 26), (602, 25), (901, 25), (1061, 24), (771, 24), (1117, 24), (777, 24), (785, 24), (1295, 24), (680, 24), (903, 24), (724, 24), (826, 24), (296, 23), (744, 23), (247, 23), (295, 23), (1178, 23), (1155, 23), (675, 23), (470, 23), (1243, 22), (369, 22), (81, 21), (858, 21), (1185, 21), (1143, 20), (879, 20), (643, 20), (1272, 20), (1209, 20), (554, 20), (931, 20), (505, 20), (835, 20), (676, 20), (580, 20), (1286, 20), (1324, 20), (715, 20), (656, 20), (1127, 20), (1051, 20), (21, 20), (1033, 20), (723, 19), (765, 19), (254, 19), (576, 19), (738, 19), (1180, 19), (760, 19), (936, 19), (670, 19), (1115, 19), (1222, 18), (716, 18), (1182, 18), (315, 18), (839, 18), (463, 18), (1186, 18), (184, 17), (1179, 17), (836, 17), (1109, 17), (743, 17), (283, 17), (704, 16), (1247, 16), (1257, 16), (485, 16), (1314, 16), (1057, 16), (1005, 16), (944, 16), (425, 16), (472, 16), (1104, 16), (996, 16), (1161, 16), (1080, 16), (834, 16), (429, 16), (577, 16), (975, 16), (490, 16), (1015, 16), (1008, 16), (1176, 15), (838, 15), (471, 15), (825, 15), (1192, 15), (1083, 15), (755, 15), (1126, 15), (1248, 15), (874, 15), (1320, 15), (898, 15), (790, 15), (1029, 14), (244, 14), (1053, 14), (1254, 14), (518, 14), (719, 14), (599, 14), (682, 14), (831, 14), (689, 14), (1184, 14), (1242, 14), (1274, 14), (997, 14), (1049, 14), (1167, 14), (371, 13), (650, 13), (1141, 13), (655, 13), (811, 13), (1079, 13), (1043, 13), (803, 13), (1157, 12), (1018, 12), (872, 12), (354, 12), (638, 12), (717, 12), (1082, 12), (991, 12), (837, 12), (1025, 12), (37, 12), (1170, 12), (805, 12), (1211, 12), (701, 12), (786, 12), (276, 12)]




[1128, 1268, 1090, 983, 833, 608, 625, 1201, 602, 901, 1061, 1117, 777, 785, 1295, 680, 903, 724, 826, 296, 771, 1178, 1155, 675, 470, 744, 247, 295, 1243, 369, 554, 858, 715, 656, 1051, 1143, 81, 879, 643, 1272, 1209, 931, 505, 835, 676, 580, 1286, 1185, 1324, 1127, 21, 1033, 765, 576, 738, 1180, 760, 670, 1222, 723, 254, 1182, 839, 936, 490, 1115, 716, 315, 743, 463, 1186, 184, 1179, 1109, 429, 283, 704, 1247, 1257, 485, 1314, 1057, 471, 1005, 944, 425, 472, 1104, 996, 836, 1161, 1126, 1080, 834, 577, 975, 1015, 1008, 244, 1176, 825, 1192, 682, 1083, 1248, 1320, 898, 790, 838, 755, 831, 1184, 874, 1242, 1274, 1029, 1053, 371, 650, 518, 719, 1141, 655, 599, 997, 1079, 1043, 1049, 1167, 503, 1254, 1211, 811, 436, 803, 1157, 1018, 872, 354, 638, 717, 1082, 991, 837, 1025, 37, 1170, 805, 701, 786]


In [ ]:
# epochs_data = agg_epochs_std
# labels = agg_labels
import sklearn
import math


entropies = []
for e in range(agg_epochs_std.shape[0]):
    u, s, vh = np.linalg.svd(agg_epochs_std[e,:,:], full_matrices=True)
    s = s/np.linalg.norm(s, ord=1)
    svd_entropy = sum([-1*(p * math.log(p)) for p in s])
    entropies.append(svd_entropy)

In [ ]:
entropies = np.asarray(entropies).reshape(-1, 1)


import sklearn 
import numpy as np
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft
from sklearn.svm import SVC


cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []




for train_idx, test_idx in cv.split(entropies):
    y_train, y_test = np.asarray(agg_labels)[train_idx], np.asarray(agg_labels)[test_idx]
    X_train = entropies[train_idx]
    X_test = entropies[test_idx]
        

    clf = sklearn.linear_model.LogisticRegression(C=1e-5)
    clf.fit(X_train, y_train)


    y_predict = clf.predict(X_test)
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(agg_labels) == np.asarray(agg_labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                                  class_balance))

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=3)
pca.fit(X)

# Neural networks

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from pyriemann.tangentspace import FGDA, TangentSpace

ind = np.linspace(0,1032, num = 900)
ind = [int(np.floor(i)) for i in ind]
epochs_ds = agg_epochs_std[:,:,ind]




cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs_ds
labels = agg_labels

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    ts = TangentSpace()
    T = ts.fit_transform(cov_X_train)
    
#     clf = sklearn.linear_model.LogisticRegression()
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(200, ), random_state=1)
    clf.fit(T, y_train)

    
    
    T_test = ts.transform(cov_X_test)
    
    y_predict = clf.predict(T_test)
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

# Polynomial regression

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from pyriemann.tangentspace import FGDA, TangentSpace

ind = np.linspace(0,1032, num = 900)
ind = [int(np.floor(i)) for i in ind]
epochs_ds = agg_epochs_std[:,:,ind]




cv = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs_ds
labels = agg_labels

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    ts = TangentSpace()
    T = ts.fit_transform(cov_X_train, y_train)
    T = rkhs(T)
    
    print(T.shape)
    
    clf = sklearn.linear_model.LogisticRegression()
    clf.fit(T, y_train)
    
    
    
    T_test = ts.fit_transform(cov_X_test)
    T_test = rkhs(T_test)
    
    y_predict = clf.predict(T_test)
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
# T = [[1,1,2],[2,1,3]]
# T = np.asarray(T)


import random, math
def polynomial(T):
    res = []
    n_samples, n_features = T.shape
    for n in range(n_samples):
        ret = []
        for k in range(100):
            ind = [random.uniform(0, 1) for i in range(n_features)]
            ind = [2 if i>0.5 else 1 for i in ind]
            s = sum([math.pow(T[n,i], ind[i]) for i in range(n_features)])
            ret.append(s)
        res.append(ret)
    return np.asarray(res)
    
    
    
# res = polynomial(T)
# res = np.asarray(res)
# print(res)

In [ ]:
import math
import numpy as np
def rkhs_helper(v):
    rkhs = []
    for i in range(len(v)-300):
        for j in range(len(v)-300):
            if i == j:
                rkhs.append(v[i]*v[j])
            else:
                rkhs.append(math.sqrt(2) * v[i]*v[j])
    return rkhs


def rkhs(T):
    ret = []
    for i in range(T.shape[0]):
        ret.append(rkhs_helper(T[i]))
    return np.asarray(ret)



# print(rkhs(np.asarray([[1,2,3],[4,5,6],[1,2,3]])))

# Quadratic Discriminant Analysis

In [ ]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  

from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from pyriemann.tangentspace import FGDA, TangentSpace

ind = np.linspace(0,1032, num = 900)
ind = [int(np.floor(i)) for i in ind]
epochs_ds = agg_epochs_std[:,:,ind]




cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs_ds
labels = agg_labels

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    ts = TangentSpace()
    T = ts.fit_transform(cov_X_train, y_train)
    
    
    clf = QuadraticDiscriminantAnalysis()
    clf.fit(T, y_train)
    
    
    
    T_test = ts.fit_transform(cov_X_test)
    
    y_predict = clf.predict(T_test)
    print(sklearn.metrics.accuracy_score(y_predict, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
tau = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45, 0.5, 0.55]
inter_adj = [np.log((1-t)*0.142857/(t*0.85714)) for t in tau]

print(inter_adj)

In [ ]:
a = [1,3,2,4,0]
n = [11,22,33,44,55]
print(np.asarray(n)[a])